In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
cmap = plt.get_cmap('inferno')
#print
plt.figure(figsize=(10,10))

#looping and creating path for every genres 
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    
    #mapping
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):    #folder names are being genereated.....still in folder
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)        
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [6]:
data.shape

(1000, 28)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[20]

array([ 1.71278873,  0.87323919,  0.40816517,  0.08352154,  0.33902875,
        0.58839083,  0.92753094, -0.42787394, -0.5235268 ,  1.36585626,
       -0.79686835,  1.08936718, -0.61568045,  1.32094307, -0.5505793 ,
        0.70015339, -0.43896028,  1.40539812, -0.96283842,  1.31995497,
       -0.66111542,  1.54508283, -0.4858743 ,  0.85343043,  0.11026233,
        0.51833271])

In [14]:
import tensorflow as tf


print(tf.__version__)



2.0.0


In [15]:
! pip install seaborn
! pip install scikit-image

In [16]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
#from keras.applications.mobilenet import MobileNet
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve
#from sklearn.metrics import auc
#import warnings
#warnings.filterwarnings("ignore")
%matplotlib inline



In [22]:
# from keras import model
# from keras import layers
# import tensorflow as tf
#from keras.models import Sequential

from keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model




# print(tf.__version__)


model = tf.keras.Sequential()
###----

model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

In [ ]:
# Epoch 1/20
# 800/800 [==============================] - 1s 1ms/step - loss: 2.1637 - acc: 0.2250
# Epoch 2/20
# 800/800 [==============================] - 0s 64us/step - loss: 1.8532 - acc: 0.3625
# Epoch 3/20
# 800/800 [==============================] - 0s 57us/step - loss: 1.6190 - acc: 0.4062
# Epoch 4/20
# 800/800 [==============================] - 0s 56us/step - loss: 1.4568 - acc: 0.5000
# Epoch 5/20
# 800/800 [==============================] - 0s 64us/step - loss: 1.3145 - acc: 0.5337
# Epoch 6/20
# 800/800 [==============================] - 0s 60us/step - loss: 1.1780 - acc: 0.6013
# Epoch 7/20
# 800/800 [==============================] - 0s 75us/step - loss: 1.0935 - acc: 0.6650
# Epoch 8/20
# 800/800 [==============================] - 0s 61us/step - loss: 1.0032 - acc: 0.6737
# Epoch 9/20
# 800/800 [==============================] - 0s 57us/step - loss: 0.9374 - acc: 0.7000
# Epoch 10/20
# 800/800 [==============================] - 0s 87us/step - loss: 0.8768 - acc: 0.7263
# Epoch 11/20
# 800/800 [==============================] - 0s 65us/step - loss: 0.8361 - acc: 0.7375
# Epoch 12/20
# 800/800 [==============================] - 0s 110us/step - loss: 0.7891 - acc: 0.7437
# Epoch 13/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.7556 - acc: 0.7563
# Epoch 14/20
# 800/800 [==============================] - 0s 74us/step - loss: 0.7123 - acc: 0.7787
# Epoch 15/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.6776 - acc: 0.7900
# Epoch 16/20
# 800/800 [==============================] - 0s 66us/step - loss: 0.6432 - acc: 0.7963
# Epoch 17/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.6190 - acc: 0.8100
# Epoch 18/20
# 800/800 [==============================] - 0s 64us/step - loss: 0.5978 - acc: 0.8188
# Epoch 19/20
# 800/800 [==============================] - 0s 60us/step - loss: 0.5537 - acc: 0.8337
# Epoch 20/20
# 800/800 [==============================] - 0s 64us/step - loss: 0.5374 - acc: 0.8463

In [ ]:

# Build Models

In [ ]:
# from sklearn.metrics import f1_score
# from time import time

In [ ]:
# def train_classifier(clf, X_train, y_train):
#     ''' Fits a classifier to the training data. '''
    
#     # Start the clock, train the classifier, then stop the clock
#     start = time()
#     clf.fit(X_train, y_train)
#     end = time()
    
#     # Print the results
#     print("Trained model in {:.4f} seconds".format(end - start))
    
# #this may not used at all    
# def predict_labels(clf, features, target):
#     ''' Makes predictions using a fit classifier based on F1 score. '''
#     #################################################F1 score astesee predict_labels(...) theke 
    
#     # Start the clock, make predictions, then stop the clock
#     start = time()
#   #  y_pred = clf.predict(features) ****Features is not defind for my dataset. if i can solve this line then
# #******************************************************{NameError: name 'y_pred' is not defined} will be 
# #######################################################solved

# ########################## ORIGIN CODE FIRST OCCURANCE OF FEATURES******************************

# #####################filepath = 'features.csv'
# ################features = pd.read_csv(filepath, index_col=0,header=[0, 1, 2], skip_blank_lines=True )

# #--------------------------------------------------------------------------------------------------

# # features is pandas read_csv dictionary 
#     end = time()
    
#     # Print and return results
#     print("Made predictions in {:.4f} seconds.".format(end - start))
#     return f1_score(target, y_pred, average='micro', pos_label = 1)



# def train_predict(clf, X_train, y_train, X_test, y_test):
#     ''' Train and predict using a classifer based on F1 score. '''
    
#     # Indicate the classifier and the training set size
#     print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
 
#     # Train the classifier
#     train_classifier(clf, X_train, y_train)
#        #untill now no problem
        
#         #### if i switched places between the train_predict and predict_labels
    
#     # Print the results of prediction for both training and testing
#     #may not work.....comment out this below section
#     print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
#     print("F1 score for val set: {:.4f}.".format(predict_labels(clf, X_val, y_val)))
#     print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [ ]:
# from pyAudioAnalysis import audioBasicIO
# from pyAudioAnalysis import ShortTermFeatures


# for g in genres:
#     pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
#     for filename in os.listdir(f'./MIR/genres/{g}'):
#         songname = f'./MIR/genres/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True, duration=5)
#         plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#         plt.axis('off');
#         plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
#         plt.clf()

# [Fs, x] = audioBasicIO.read_audio_file("sample.wav")
# F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)



In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# #from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

# # TODO: Initialize the three models
# clf_A = DecisionTreeClassifier(random_state=10, max_depth =4)
# clf_B = SVC()
# clf_C = LogisticRegression()
# clf_D = RandomForestClassifier(random_state=10, max_depth=30, n_estimators=300, min_samples_leaf=6, min_impurity_decrease=0.0002,
#                      class_weight='balanced')

# for clf in [clf_A, clf_B, clf_C, clf_D]:
#     print("\n{}: \n".format(clf.__class__.__name__))
#     train_predict(clf, X_train, y_train, X_test, y_test)   # X_val turned to X_test, y_val to y_test



In [ ]:
### Classification of Music into different Genres using Keras


In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler#Keras
import keras

In [ ]:
###create a CSV file

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
###calculate all the features.

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    #for filename in os.listdir(f'./genres/{g}'):              ###############Eroor****FileNotFoundError: [WinError 3] The system cannot find the path specified: './genres/blues'
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)   ###errror****FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\SCC\\genres\\blues\\blues.00000.wav'
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
            
            
            
            #ran with out an error

In [ ]:
data = pd.read_csv('data.csv')
data.head()

In [ ]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

In [ ]:
data.iloc[:, : -1]

In [ ]:
#######np.array(data.iloc[:, : -1] , dtype = float) ##########here's the error

####np.array(data.iloc[:, : -1] , dtype.astype = float)

np.array(data.iloc[:, : -1])




In [ ]:
scaler = StandardScaler()


#convertion
x = np.array(data.iloc[:, :-1])
Y = x.astype(np.float)


X = scaler.fit_transform(Y)

In [ ]:
# convert X into dataframe
X_pd = pd.DataFrame(data=x)
# replace all instances of URC with 0 
X_replace = for item in X_pd:
                
                X_pd.replace(f'rock.000{num}.wav',0, regex=True)

# # convert it back to numpy array
X_np = X_replace.values
# # set the object type as float
X_fa = X_np.astype(dtype = float)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))